## Calculate eddy currents (for examples of transient patterns)

In [1]:
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np
import sys

In [5]:
from coils.eddycurrents0 import diagonalize, transient_amplitudes, SplitBoxRoom, UNIT
from coils.coils import ScalarCoilPlate, DipoleSet
import pyfunk as pf

Define some helper classes and functions

In [6]:
def inv_step_response(system, coil, field_p, field_dir, length, mode_selection = None):
    """Get inverse step response of an eddy-current system to pulsed coil.
    
    Arguments:
        system: the eddy current system (CoilSystem object)
        coil: an CoilSystem object representing the coil
        field_p: the point at which the field is measured
        field_dir: a unit vector in the direction of the field measurement
        length: length of the inverse step response in seconds
    """
    
    def mode_responses():
        from functools import partial
        fun = lambda t, tau: np.exp(-t/tau)
        for lambda_, tau in zip(system.lambdas, system.tau):
            yield pf.Function(partial(fun, tau=tau), (0.0, length))
        
    basis = FunctionBasis(mode_responses())
    
    coil_c = coil.mutual_inductances(system).flatten()
    field_c = system.generated_fields(field_p).T.dot(field_dir.flatten())
    
    mode_input = system.V.T.dot(coil_c) / system.lambdas
    mode_output = system.V.T.dot(field_c)
    
    return basis.get_waveform(mode_input * mode_output)

def calc_response(pulse, resp_function):
    """Get system response to pulse, based on given inverse step response."""
    return -pulse.differentiate().convolve(resp_function)

class LinearCombination(pf.Waveform):
    """A waveform that is a linear combination of other waveforms."""
    
    def __init__(self, functions, coefficients):
        self.coefficients = coefficients
        self.functions = functions
        self.bound_tags = \
            tuple((max((b.bound_tags[0] for b in self.functions), key = lambda x: x.value),
                   min((b.bound_tags[1] for b in self.functions), key = lambda x: x.value)))

    def sample(self, positions):
        return np.sum([bf.sample(positions) * c 
                           for bf, c in zip(self.functions, self.coefficients)], axis = 0)

class FunctionBasis(object):
    """A waveform function basis"""
    def __init__(self, basis_waveforms):
        self.functions = list(basis_waveforms)
    
    def get_waveform(self, coefficients):
        """Get the waveform based on a coefficient or 'coordinate' vector."""
        return LinearCombination(self.functions, coefficients)


Define the shielded room and coils for computational modeling.

In [7]:
# Some ft units are used to define this, because it is how the Berkeley
# shielded room was built, but everything gets quickly converted to SI units
room = SplitBoxRoom(detail_scale = UNIT.FOOT,
                    thickness = 10e-3)
diagonalize(room)

# polarizing coil (modeled as a dipole)
# 2022-06: tämän suuntaa on myös helppo kääntää t. Koos

p_n = 240.0
p_r = 0.19
p_I = 200.0

p_center = np.array([0,0,-3.0 * 0.0254]) # position of coil center
p_dipole = np.array([0,0,1.]) * p_n * np.pi * p_r**2 # dipole moment
p_coil = DipoleSet(p_center, p_dipole)

# DynaCAN coil (larger square)
c_side = 1.85
c_turns = 30
c_res = 100
c_R = 30./40*1.2
c_L = (30./40)**2*10e-3
c_center = -4*UNIT.FOOT + 1.125
c_coil = ScalarCoilPlate(c_center,
                         (np.array([c_side,0,0]), np.array([0,c_side,0])),
                         np.ones((1,c_res,c_res)) * c_turns,
                         label = "dynacan_coil")

Prepare to test by calculating ISRs for the two coils and a magnetic field measurement.

In [8]:
field_p = np.array([0,0,0]) # measurement point
field_dir = np.array([0,0,1.]) # unit vector for field component

isr_length = 500e-3
# calculate inverse step responses for both coils
p_isr = inv_step_response(room, p_coil, 
                          field_p, field_dir, isr_length).to_samples()
c_isr = inv_step_response(room, c_coil, 
                          field_p, field_dir, isr_length).to_samples()

.....................

Try computing the eddy currents caused by a polarizing pulse.

In [13]:
p_ramp_time = 11.6e-3
p_pulse = p_I * pf.RampedBox(1.0, 
                             (-0.2, -p_ramp_time), 
                             (10e-3, p_ramp_time), 
                             (pf.LinearRamp, pf.QuarterCosineRamp))

# calculate response from p coil pulse
p_trans = calc_response(p_pulse, p_isr)

In [14]:
plt.figure("Polarizing pulse response")
p_trans.plot()
plt.xlabel("Time [s]")
plt.ylabel("Magnetic field [T]")
plt.tight_layout()

Try with some random pulse waveform fed into the DynaCAN coil

In [15]:
pulse_duration = 100e-3

n_basis = 20
basis_wfs = [pf.Function(lambda x, n = n: np.sin(-x/pulse_duration * n * np.pi), (-pulse_duration, 0))
                 for n in range(1, n_basis + 1)]
basis = FunctionBasis(basis_wfs)



In [16]:
# random waveform times 1 ampere
rand_coefficients = 2.0*(np.random.rand(n_basis)-0.5)
rand_coefficients[0:2] = 0 # Get some more interesting transients
randompulse = 1.0 * basis.get_waveform(rand_coefficients) 
c_trans_from_random = calc_response(randompulse, c_isr).clip(0.0, isr_length - pulse_duration)


In [17]:
plt.figure("Random pulse response")
plt.subplot(2,1,1)
randompulse.plot()
plt.xlabel("Time [s]")
plt.ylabel("Current [A]")
plt.subplot(2,1,2)
c_trans_from_random.plot()
plt.xlabel("Time [s]")
plt.ylabel("Magnetic field [T]")
plt.tight_layout()

The result can be sampled at a number of time points by calling `c_trans_from_random.sample(array_with_time_points)`.

The field can be computed at different points in space with separate ISRs corresponding to each point. Let's load up the locations of the sensors.

In [18]:
array120 = dict(np.load("array120_trans_newnames.npz"))

In [19]:
array120["MEG0111"]

array([[-0.0127    , -0.186801  , -0.98232698, -0.1066    ],
       [ 0.0057    , -0.98240298,  0.18674099,  0.0464    ],
       [-0.99990302, -0.0033    ,  0.013541  , -0.0604    ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

This is a 4x4 transformation matrix for transforming a point $[x,y,z,1]$ from the coordinate system of the individual sensor into the laboratory coordinate system. Hence, the position (center) of the sensor is given by matrix[:3, 3] and the direction of the $z$ axis of the sensor is given by matrix[:3, 2].

Each magnetometer measures the field component along its own $z$ axis.

In [20]:
#TODO: compute an ISR for each sensor and use that to obtain time traces for each sensor

We can also plot values as colors on the helmet, as shown below. You could plot e.g. some some component in multidimensional signal space, or a binary value of whether the some specific thing is going on with the channel.

In [ ]:
import viz
from mayavi import mlab

viz.plot_sensor_data(["MEG0241"],
                     [0.0,4.0,10.0])
mlab.show()